In [1]:
import os
import tensorflow as tf
import pickle
from utils import DataLoader
import re
import numpy as np
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, "data")
session = tf.InteractiveSession()

In [2]:
max_seq_length = 100
batch_size = 10
rnn_size = 50
W_inputs = tf.constant(np.random.random((rnn_size, rnn_size)), dtype="float32")
W_pointerRNN = tf.constant(np.random.random((rnn_size, rnn_size)), dtype="float32")
v = tf.constant(np.random.random((rnn_size, 1)), dtype="float32")
W_inputs_tiled = tf.tile(tf.expand_dims(W_inputs, 0), [batch_size, 1, 1])
#W_pointerRNN_tiled = tf.tile(tf.expand_dims(W_pointerRNN, 0), [batch_size, 1, 1])
v_tiled = tf.tile(tf.expand_dims(v, 0), [batch_size, 1, 1])

In [3]:
inputs = tf.constant(np.random.random((batch_size, max_seq_length, rnn_size)), dtype="float32")
def compute_lengths(inputs):
    used = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2))
    lengths = tf.reduce_sum(used, reduction_indices=1)
    lengths = tf.cast(lengths, tf.int32) #lengths must be integers
    return lengths
seq_lengths = compute_lengths(inputs)

In [4]:
#TODO: use attention-pooling over the question representation for the initialize hidden vector of pointer network
def loop_fn(time, prev_output, prev_state, prev_loop_state):
    print('start loop', time)
    next_output = prev_output
    if prev_output is None:
        next_state = cell.zero_state(batch_size, tf.float32)
    else:
        next_state = prev_state
    next_loop_state = None
    def get_next_input():
        print("computing logits")
        print("prev_output", prev_output)
        if prev_output is None:
            logits = tf.matmul(tf.tanh(tf.matmul(inputs, W_inputs_tiled)), v_tiled) #batch_size x seq_length x rnn_size
        else:
            print("i have prev output now")
            logits = tf.matmul(
                             tf.tanh(tf.matmul(inputs, W_inputs_tiled) + 
                                     tf.tile(tf.expand_dims(tf.matmul(prev_output, W_pointerRNN), 1), [1, max_seq_length, 1]))
                             , v_tiled) #batch_size x seq_length x rnn_size
        print("computing probs")
        predicted_probs = tf.nn.softmax(logits, dim=1) #a
        print("predicted_probs")
        weighted_input = tf.reduce_sum(tf.multiply(predicted_probs, inputs), axis=1) #c #batch_size x rnn_size
        print("returning weighted_input")
        return weighted_input #weighted input is next input
    print("1")
    elements_finished = (time >= seq_lengths) #this operation produces boolean tensor of [batch_size] defining if corresponding sequence has ended
    print("2")
    finished = tf.reduce_all(elements_finished) #AND operation over all batches. True if all batches finished.
    time += 1
    print("3")
    next_input = tf.cond(finished, lambda: tf.zeros([batch_size, rnn_size], dtype=tf.float32), get_next_input)
    print("4")
    return elements_finished, next_input, next_state, next_output, next_loop_state

In [5]:
with tf.variable_scope(tf.get_variable_scope(), reuse=False):
    cell = tf.contrib.rnn.LSTMCell(rnn_size)
    outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
    outputs = outputs_ta.stack()
    #predicted_idx = tf.argmax(predicted_probs) #p

start loop Tensor("rnn/Const:0", shape=(), dtype=int32)
1
2
3
computing logits
prev_output None
computing probs
predicted_probs
returning weighted_input
4
start loop Tensor("rnn/while/add:0", shape=(), dtype=int32)
1
2
3
computing logits
prev_output Tensor("rnn/while/lstm_cell/mul_2:0", shape=(10, 50), dtype=float32)
i have prev output now
computing probs
predicted_probs
returning weighted_input
4


In [9]:
tf.shape(outputs)

<tf.Tensor 'Shape_2:0' shape=(3,) dtype=int32>

In [11]:
outputs.get_shape().as_list()

[None, 10, 50]

In [ ]:
inputs = tf.constant(np.random.random((batch_size, max_seq_length, rnn_size)), dtype="float32")
inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_seq_length)
inputs_ta = inputs_ta.unstack(inputs)

In [ ]:
inputs_ta.size().eval()

In [ ]:
inputs_ta.read(3)

In [ ]:
inputs = tf.constant(np.random.random((max_seq_length, batch_size, rnn_size)), dtype="float32")
inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_seq_length)
inputs_ta = inputs_ta.unstack(inputs)

In [ ]:
inputs_ta.read(3)

In [ ]:
cell = tf.contrib.rnn.LSTMCell(rnn_size, reuse=True)
outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()
outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()
predicted_idx = tf.argmax(self.predicted_probs) #p

In [ ]:
#TODO: use attention-pooling over the question representation for the initialize hidden vector of pointer network
def loop_fn(time, prev_output, prev_state, prev_loop_state):
    print('start loop')
    next_output = prev_output
    if prev_output is None:
        next_cell_state = cell.zero_state(batch_size, tf.float32)
    else:
        next_cell_state = prev_state
    next_loop_state = None
    def get_next_input():
        print("computing logits")
        print(inputs_ta.read(time))
        print(prev_output)
        if prev_output is None:
            logits = tf.matmul(v, tf.tanh(tf.matmul(inputs_ta.read(time), W_inputs)))
        else:
            logits = tf.matmul(tf.tanh(tf.matmul(inputs_ta.read(time), W_inputs) + tf.matmul(prev_output, W_pointerRNN)), v) #s #at each step network has access to all paragraph inputs, plus the previous RNN outputs
        print("computing probs")
        predicted_probs = tf.nn.softmax(logits) #a
        print("computing weighted_input")
        weighted_input = tf.reduce_sum(tf.multiply(predicted_probs, inputs)) #c
        return weighted_input #weighted input is next input
    print("1")
    elements_finished = (time >= seq_lengths) #this operation produces boolean tensor of [batch_size] defining if corresponding sequence has ended
    print("2")
    finished = tf.reduce_all(elements_finished) #AND operation over all batches. True if all batches finished.
    time += 1
    print("3")
    next_input = tf.cond(finished, lambda: tf.zeros([batch_size, rnn_size], dtype=tf.float32), get_next_input)
    print("4")
    return elements_finished, next_input, next_state, next_output, next_loop_state

In [ ]:
inputs = tf.constant(np.random.random((batch_size, max_seq_length, rnn_size)))
def compute_lengths(inputs):
    used = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2))
    lengths = tf.reduce_sum(used, reduction_indices=1)
    lengths = tf.cast(lengths, tf.int32) #lengths must be integers
    return lengths
seq_lengths = compute_lengths(inputs)
seq_lengths.eval()
def loop_fn(time, cell_output, cell_state, loop_state):
    print("start loop")
    emit_output = cell_output  # == None for time == 0
    if cell_output is None:  # time == 0
        next_cell_state = cell.zero_state(batch_size, tf.float32)
    else:
        next_cell_state = cell_state
    print("1")
    elements_finished = (time >= seq_lengths)
    print("2")
    finished = tf.reduce_all(elements_finished)
    print("3")
    next_input = tf.cond(
        finished,
        lambda: tf.zeros([batch_size, rnn_size], dtype=tf.float32),
        lambda: inputs_ta.read(time))
    print("4")
    next_loop_state = None
    print("5")
    print("elements_finished", elements_finished)
    print("next_input", next_input)
    print("next_cell_state", next_cell_state)
    print("next_loop_state", next_loop_state)
    return elements_finished, next_input, next_cell_state, emit_output, next_loop_state

max_seq_length = 100
batch_size = 10
rnn_size = 50
W_inputs = tf.constant(np.random.random((rnn_size, rnn_size)))
W_pointerRNN = tf.constant(np.random.random((rnn_size, rnn_size)))
v = tf.constant(np.random.random((1, rnn_size)))
inputs = tf.constant(np.random.random((max_seq_length, batch_size, rnn_size)))
inputs_ta = tf.TensorArray(dtype=tf.float32, size=max_seq_length)
inputs_ta = inputs_ta.unstack(inputs)
cell = tf.contrib.rnn.LSTMCell(rnn_size, reuse=True)
outputs_ta, final_state, _ = tf.nn.raw_rnn(cell, loop_fn)
outputs = outputs_ta.stack()